In [ ]:
import pathlib as pl
import numpy as np
import matplotlib.pyplot as plt
import flopy
import flopy.mf6 as fp6

# Example 1 Model - Build and run the (serial) model with FloPy

In [ ]:
# model name and simulation workspace
name = "ex1"
ws = pl.Path("working/single")

# Problem Dimensions

## Temporal Discretization

In [ ]:
# create a list with the (perlen, nstp, tsmult) for each stress period
tdis_data = [
    (0., 1, 1.0),
    (300000., 1, 1.0),
    (36500., 10, 1.5),
    (300000, 1, 1.0),
]
nper = len(tdis_data)

## Spatial Discretization

In [ ]:
# set nlay, nrow, and ncol
nlay, nrow, ncol = 3, 21, 20

In [ ]:
# set size of model in x- and y-directions
xlen, ylen = 10000., 10500.

In [ ]:
# calculate delr (cell width along each row [dx]) and delc (cell width along each column [dy])
delc = ylen / nrow
delr = xlen / ncol
delr, delc

In [ ]:
# set top of the model and the bottom of each layer
top = 400.
botm = [220., 200., 0]

## Hydraulic Properties

In [ ]:
# set the K in the horizontal and vertical directions
K11 = [50., 0.01, 200.]
K33 = [10., 0.01, 20.]

## Storage properties

In [ ]:
# set specific storage and specific yield - constant for each cell
Ss, Sy = 0.0001, 0.1

## Boundary Condition data

In [ ]:
# set the stage in the river
H_east = 320.

In [ ]:
# set recharge rate
recharge = 0.005

# Build the model

The model will have the following objects and packages:

* simulation (sim) object
* temporal discretization (TDIS) package
* iterative model solution (IMS) package
* groundwater flow model (gwf) object
* structured discretization (DIS) package
* node property flow (NPF) package
* storage (STO) package
* initial conditions (IC) package
* output control (OC) package
* recharge (RCH) package
* constant head (CHD) package
* well (WEL) package

## MODFLOW Simulation files

In [ ]:
# create the sim object using MFSimulation
sim = fp6.MFSimulation(sim_name=name, sim_ws=ws)

In [ ]:
# create the tdis package using ModflowTdis
tdis = fp6.ModflowTdis(sim, nper=nper, perioddata=tdis_data, time_units="days")

In [ ]:
# create the ims package using ModflowIms
# set complexity="simple" and outer_dvclose and inner_dvclose to 1e-6
ims = fp6.ModflowIms(sim, complexity="simple", print_option="all", outer_dvclose=1e-6, inner_dvclose=1e-6)

## Groundwater flow model

In [ ]:
# create the gwf object using ModflowGwf
# set save_flows=True
gwf = fp6.ModflowGwf(sim, modelname=name, save_flows=True)

In [ ]:
# create the structured discretization file using ModflowGwfdis
dis = fp6.ModflowGwfdis(gwf, nlay=nlay, nrow=nrow, ncol=ncol, delr=delr, delc=delc, top=top, botm=botm, length_units="meters")

In [ ]:
# look at the gwf modelgrid
gwf.modelgrid

In [ ]:
# plot the model grid
gwf.modelgrid.plot()

In [ ]:
# create the npf package using ModflowGwfnpf
# set icelltype=1 and save_specific_discharge=True
npf = fp6.ModflowGwfnpf(gwf, icelltype=1, save_specific_discharge=True, k=K11, k33=K33)

In [ ]:
# create the sto package using ModflowGwfsto
# set iconvert=1, steady_state={0:True, 3:True}, and transient={2:True}
sto = fp6.ModflowGwfsto(gwf, iconvert=1, ss=Ss, sy=Sy, steady_state={0:True, 3:True}, transient={2:True})

In [ ]:
# create the ic package using fp6.ModflowGwfic
# set strt=top
ic = fp6.ModflowGwfic(gwf, strt=top)

In [ ]:
# create the oc package
# set head_filerecord=f"{name}.hds", budget_filerecord=f"{name}.cbc", and saverecord={0: [("head", "all"), ("budget", "all")]}
oc = fp6.ModflowGwfoc(gwf, head_filerecord=f"{name}.hds", budget_filerecord=f"{name}.cbc", saverecord={0: [("head", "all"), ("budget", "all")]})

## Boundary conditions

### Groundwater recharge

In [ ]:
# create the rch package using ModflowGwfrcha
rch = fp6.ModflowGwfrcha(gwf, recharge=recharge)

### Constant head

In [ ]:
# create a list of chd cells representing the river on the east side of the model
chd_spd = [(0, i, ncol-1, H_east) for i in range(nrow)]

In [ ]:
# create the chd package using ModflowGwfchd
chd = fp6.ModflowGwfchd(gwf, stress_period_data=chd_spd)

### Wells

In [ ]:
# create a dictionary of well pumping rates for each stress period
well_spd = {
    0: [],
    1: [(0, 10, 9, -75000.)],
    2: [
        (0, 10, 9, -75000.),
        (2, 12, 4, -100000.)
    ],
    3: [
        (0, 10, 9, -75000.),
        (2, 12, 4, -100000.)
    ],
}

In [ ]:
# alternative way to specify well data
well_spd = {
    1: [(0, 10, 9, -75000.)],
    2: [
        (0, 10, 9, -75000.),
        (2, 12, 4, -100000.)
    ],
}

In [ ]:
# create the wel package using ModflowGwfwel
wel = fp6.ModflowGwfwel(gwf, stress_period_data=well_spd)

## Write the model files and run the simulation

In [ ]:
# write the model input files
sim.write_simulation()

In [ ]:
# run the model
sim.run_simulation()

## Get model output

Get the model heads and specific discharge using `gwf.output.` flopy functionality.

### Heads

In [ ]:
# determine what output are available
gwf.output.methods()

In [ ]:
# what times are in the head file
gwf.output.head().get_times()

In [ ]:
# what (time step, stress period) are in the head file
gwf.output.head().get_kstpkper()

In [ ]:
# Set a select (time step, stress period) or totim that can be used for all output extractions
kstpkper = (9,2)

In [ ]:
# Get the groundwater heads for a select (time step, stress period) or totim
# Name the variable head
head = gwf.output.head().get_data(kstpkper=kstpkper)

## Specific discharge

In [ ]:
# determine what output data is in the budget file
gwf.output.budget().get_unique_record_names()

In [ ]:
# get the specific discharge
spdis = gwf.output.budget().get_data(text="DATA-SPDIS", kstpkper=kstpkper)[0]

In [ ]:
# list what is in specific discharge data
spdis

In [ ]:
# convert specific discharge into qx, qy, and qz vectors using flopy.utils.postprocessing.get_specific_discharge()
# name the variables qx, qy, and qz
qx, qy, qz = flopy.utils.postprocessing.get_specific_discharge(spdis, gwf)

## Plot model results

### Create a Map for all three layers

In [ ]:
fig = plt.figure(figsize=(15,7))

for k in (0, 1, 2):
    ax = fig.add_subplot(1, 3, k + 1)
    pmv = flopy.plot.PlotMapView(model=gwf, layer=k, ax=ax)
    hp = pmv.plot_array(head)
    pmv.plot_grid()
    pmv.plot_vector(qx, qy)
    cb = plt.colorbar(hp, shrink=0.75, orientation="horizontal")
    ax.set_title(f"Layer {k + 1}")


### Create a Cross-Section

In [ ]:
# set the row to create a cross-section along
xsec_row = 12

In [ ]:
xsect = flopy.plot.PlotCrossSection(model=gwf, line={"Row": xsec_row})
csa = xsect.plot_array(head, head=head)
xsect.plot_grid()
quiver = xsect.plot_vector(
    qx,
    qy,
    qz,
    head=head,
    hstep=1,
    normalize=True,
    color="black",
    scale=30,
    headwidth=3,
    headlength=3,
    headaxislength=3,
    zorder=10,
)
xsect.ax.set_title(f"Cross-section row {xsec_row + 1}")
cb = plt.colorbar(csa, shrink=0.75);

In [ ]:
# set the row to create a cross-section along
xsec_row = 10

In [ ]:
xsect = flopy.plot.PlotCrossSection(model=gwf, line={"Row": xsec_row})
csa = xsect.plot_array(head, head=head)
xsect.plot_grid()
quiver = xsect.plot_vector(
    qx,
    qy,
    qz,
    head=head,
    hstep=1,
    normalize=True,
    color="black",
    scale=30,
    headwidth=3,
    headlength=3,
    headaxislength=3,
    zorder=10,
)
xsect.ax.set_title(f"Cross-section row {xsec_row + 1}")
cb = plt.colorbar(csa, shrink=0.75);